In [2]:
import pandas as pd
import numpy as np


!conda install -c conda-forge Bs4 --yes
from bs4 import BeautifulSoup
import html5lib
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!conda install -c conda-forge geopy --yes  
from  geopy.geocoders  import Nominatim  # To convert  addresses into latitude and longitude values

!conda install -c conda-forge/label/cf202003 geocoder --yes
import geocoder

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - bs4


The following packages will be UPDATED:

  ca-certificates    conda-forge/label/cf202003::ca-certif~ --> conda-forge::ca-certificates-2020.4.5.1-hecc5488_0
  certifi            conda-forge/label/cf202003::certifi-2~ --> conda-forge::certifi-2020.4.5.1-py36h9f0ad1d_0


Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geocoder


The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    conda-forge::ca-certificates-2020.4.5~ --> conda-forge/label/cf202003::ca-certificates-2019.11.28-hecc5488_0
  certifi            conda-forge::certifi-2020.4.5.1-py36h~ --> conda-forge/lab

In [46]:
data = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', flavor='html5lib')
table = data[0]

In [47]:
print(table.shape)
table.head()

(180, 3)


,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [5]:
# Dropping Nan values Neighborhoods  as far as "Not assigned" Boroughs
table = table[table['Neighborhood'].notnull()].reset_index(drop=True)

# Looking for double Postal Code values, and concatenate their Neighborhood with the separation " , "
#table = table.groupby(['Postal Code', 'Borough'], as_index=False).agg({'Neighborhood': ','.join})

print("The shape of our data is {}".format(table.shape))
table.head()

The shape of our data is (103, 3)


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [6]:
# Coordinates of Toronto , Ontario
geo = geocoder.arcgis('Toronto, Ontario')
latitude  = geo.latlng[0]
longitude = geo.latlng[1]

# Getting Latitudes and Longitudes coordinate for each postal code
latitudes  = []
longitudes = []
for postal_code in table['Postal Code'] :
    lat_long = None
    #while (lat_long is None):
    geo  = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
    lat_long = geo.latlng
    latitudes.append(lat_long[0])
    longitudes.append(lat_long[1])
print(latitude,longitude)

43.648690000000045 -79.38543999999996


In [7]:
table['Latitude']  = latitudes
table['Longitude'] = longitudes
table.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752935,-79.335641
1,M4A,North York,Victoria Village,43.728102,-79.311890
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390


In [8]:
Toronto_map = folium.Map(location =[latitude, longitude], zoom_start =10.3)

for lng,lat,bor,nei in zip(table['Longitude'], table['Latitude'], table['Borough'],table['Neighborhood']):    
    label = '{}, {}'.format(nei, bor)
    label = folium.Popup(label, parse_html =True)
    folium.CircleMarker(
    [lat,lng],
    radius = 5,
    popup = label,
    fill = True,
    color = 'darkblue',
    fill_color = '#000080',
    fill_opacity=0.7,
    parse_html = False).add_to(Toronto_map)

Toronto_map

# <center> Exploring Neighborhoods around Toronto, Ontario </center>

##### Foursquare Credentials Api and Version

In [9]:
CLIENT_ID = 'OALK4BD0CEZFB5PNPR12U4P4ZSYCUWGJUXXWLBTJFIHDSQR2'
CLIENT_SECRET = 'JHMTADIJMVQPNOTSMW5VME4VGW2SKZMS1NWDBSGXO2HBC33D'
VERSION = '20200525'

Let first explore the first neighborhood in our data, and get the top 100  Venues around within a radius of 500 meters

In [10]:
LAT = table.loc[0,'Latitude']
LNG = table.loc[0,'Longitude']
first_nei = table.loc[0,'Neighborhood']
print("""The first Neighborhood we are exploring is {} with
        latitude {} and longitude {}""".format(first_nei, LAT, LNG) )

The first Neighborhood we are exploring is Parkwoods with
        latitude 43.75293455500008 and longitude -79.33564142299997


Foursquare-API request

In [14]:
import requests

LIMIT = 100
RADIUS = 500
url ="https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&ll={},{}&v={}&limit={}&\
        #radius={}".format(CLIENT_ID,CLIENT_SECRET,LAT,LNG,VERSION,LIMIT,RADIUS )

result = requests.get(url).json()

In [15]:
venues = result['response']['groups'][0]['items']
venues = pd.json_normalize(venues)
venues.head()

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,...,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.neighborhood,venue.location.crossStreet,venue.venuePage.id
0,e-0-4bd4846a6798ef3bd0c5618d-0,0,"[{'summary': 'This spot is popular', 'type': '...",4bd4846a6798ef3bd0c5618d,Donalda Golf & Country Club,12 Bushbury Dr,43.752816,-79.342741,"[{'label': 'display', 'lat': 43.75281596740471...",571,...,Don Mills,ON,Canada,"[12 Bushbury Dr, Don Mills ON M3A 2Z7, Canada]","[{'id': '4bf58dd8d48988d1e6941735', 'name': 'G...",0,[],NaN,NaN,NaN
1,e-0-4b8991cbf964a520814232e3-1,0,"[{'summary': 'This spot is popular', 'type': '...",4b8991cbf964a520814232e3,Allwyn's Bakery,81 Underhill drive,43.759840,-79.324719,"[{'label': 'display', 'lat': 43.75984035203157...",1167,...,Toronto,ON,Canada,"[81 Underhill drive, Toronto ON M3A 1Z5, Canada]","[{'id': '4bf58dd8d48988d144941735', 'name': 'C...",0,[],Parkwoods - Donalda,NaN,NaN
2,e-0-4ccec87654f0b1f7f32824ca-2,0,"[{'summary': 'This spot is popular', 'type': '...",4ccec87654f0b1f7f32824ca,Galleria Supermarket,865 York Mills Rd.,43.753520,-79.349518,"[{'label': 'display', 'lat': 43.75352001200256...",1117,...,Toronto,ON,Canada,"[865 York Mills Rd. (at Upjohn Rd.), Toronto O...","[{'id': '52f2ab2ebcbc57f1066b8b46', 'name': 'S...",0,[],NaN,at Upjohn Rd.,NaN
3,e-0-4b0aed06f964a520202a23e3-3,0,"[{'summary': 'This spot is popular', 'type': '...",4b0aed06f964a520202a23e3,Island Foods,1310 Don Mills Rd.,43.745866,-79.346035,"[{'label': 'display', 'lat': 43.74586607398898...",1147,...,Toronto,ON,Canada,"[1310 Don Mills Rd., Toronto ON, Canada]","[{'id': '4bf58dd8d48988d144941735', 'name': 'C...",0,[],NaN,NaN,NaN
4,e-0-4b8ec91af964a520053733e3-4,0,"[{'summary': 'This spot is popular', 'type': '...",4b8ec91af964a520053733e3,Graydon Hall Manor,185 Graydon Hall Drive,43.763923,-79.342961,"[{'label': 'display', 'lat': 43.76392256055678...",1357,...,Toronto,ON,Canada,"[185 Graydon Hall Drive, Toronto ON M3A 3B4, C...","[{'id': '4bf58dd8d48988d171941735', 'name': 'E...",0,[],NaN,NaN,52496423


In [16]:
venues.loc[0,'venue.categories']

[{'id': '4bf58dd8d48988d1e6941735',
  'name': 'Golf Course',
  'pluralName': 'Golf Courses',
  'shortName': 'Golf Course',
  'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/golfcourse_',
   'suffix': '.png'},
  'primary': True}]

In [17]:
def get_categories(row):
    try:
        cat_list = row['categories']
    except :
        cat_list = row['venue.categories']
    if len(cat_list) == 0:
        return None
    else:
        return cat_list[0]['name']

In [18]:
venues.shape

(100, 22)

In [19]:
#filtered_columns =[Name, Categories, Latitude, Longitu]
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =venues.loc[:, filtered_columns]
nearby_venues.loc[:,'venue.categories'] =  venues.apply(get_categories, axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,Donalda Golf & Country Club,Golf Course,43.752816,-79.342741
1,Allwyn's Bakery,Caribbean Restaurant,43.759840,-79.324719
2,Galleria Supermarket,Supermarket,43.753520,-79.349518
3,Island Foods,Caribbean Restaurant,43.745866,-79.346035
4,Graydon Hall Manor,Event Space,43.763923,-79.342961


In [20]:
nearby_venues.tail()

,name,categories,lat,lng
95,China Cottage,Hakka Restaurant,43.759097,-79.310419
96,Tilley Endurables,Clothing Store,43.727033,-79.342926
97,DAVIDsTEA,Tea Room,43.777593,-79.345089
98,LCBO,Liquor Store,43.736323,-79.346115
99,Foodland - Toronto,Grocery Store,43.773951,-79.340148


#### Let explore all the neighborhoods in Toronto City

In [21]:
def get_Nearby_venues(name, latitude, longitude, radius =500):
    """
    Get the most 100 venues around a given neighborhood name and a given couple (lat,long) in a radius 
    of 500 meters"""
    LAT = latitude
    LNG = longitude
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        LAT, 
        LNG, 
        radius, 
        LIMIT)
    try :
        result = requests.get(url).json()
        venues = result['response']['groups'][0]['items']
        venues = pd.json_normalize(venues)
        filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
        nearby_venues =venues.loc[:, filtered_columns]
        nearby_venues.loc[:,'venue.categories'] =  venues.apply(get_categories, axis=1)
        #nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
    except :
        filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
        nearby_list = [[None,None,None,None]]
        nearby_venues = pd.DataFrame(nearby_list, columns=filtered_columns)
        print('there is a problem')
    #print(nearby_venues)
    return nearby_venues

In [22]:
# Get a Loop into table for each neighborhood and corresponding latitude and longitude
# with the radius default value 500 meters
compt =0 
temp = []
for bor,name,lat,long in zip(table["Borough"],table['Neighborhood'], table['Latitude'], table['Longitude']):
    nearby_venues = get_Nearby_venues(name,lat,long)
    for i in range(nearby_venues.shape[0]) :
        temp.append([bor,name, lat,long,nearby_venues.loc[i,'venue.name'],nearby_venues.loc[i,'venue.categories']\
                      ,nearby_venues.loc[i,'venue.location.lat'],nearby_venues.loc[i,'venue.location.lng']] ) 
    compt = compt +1
    #if compt == 1 :
    #    break
    #except :
    #temp.append([(name, lat, long,None,None,None,None)])
# Create a DataFrame with the extracted venues for each Neighborhood in Toronto

filtered_columns = ['Borough Name','Neighborhood Name', 'Neighborhood Latitude','Neighborhood Longitude','Venue Name', 'Venue Categorie', 'Venue Latitude', 'Venue Longitude']
Toronto_nei_venues = pd.DataFrame(temp, columns =filtered_columns)

there is a problem
there is a problem


The above output suggest that there are two neighborhood with an incorrect request's response.
Let see the five first rows

In [23]:
Toronto_nei_venues.head()

,Borough Name,Neighborhood Name,Neighborhood Latitude,Neighborhood Longitude,Venue Name,Venue Categorie,Venue Latitude,Venue Longitude
0,North York,Parkwoods,43.752935,-79.335641,Brookbanks Park,Park,43.751976,-79.332140
1,North York,Parkwoods,43.752935,-79.335641,Variety Store,Food & Drink Shop,43.751974,-79.333114
2,North York,Parkwoods,43.752935,-79.335641,649 Variety,Convenience Store,43.754513,-79.331942
3,North York,Victoria Village,43.728102,-79.311890,Tim Hortons,Coffee Shop,43.725517,-79.313103
4,North York,Victoria Village,43.728102,-79.311890,Portugril,Portuguese Restaurant,43.725819,-79.312785


The shape of the explored neighborhoods dataframe is 

In [24]:
Toronto_nei_venues.shape

(2276, 8)

Let see the neighborhoods without any proper venue response and then drop them from the table data

In [25]:
Toronto_nei_venues[Toronto_nei_venues['Venue Name'].isnull()]

,Borough Name,Neighborhood Name,Neighborhood Latitude,Neighborhood Longitude,Venue Name,Venue Categorie,Venue Latitude,Venue Longitude
262,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.785779,-79.157368,None,None,NaN,NaN
1926,Scarborough,Upper Rouge,43.834768,-79.204101,None,None,NaN,NaN


In [26]:
Toronto_nei_venues = Toronto_nei_venues[~(Toronto_nei_venues['Venue Name'].isnull())]

Let visualize how many venues we have for each neighborhoods

In [27]:
Toronto_nei_venues.groupby('Neighborhood Name').count()

,Borough Name,Neighborhood Latitude,Neighborhood Longitude,Venue Name,Venue Categorie,Venue Latitude,Venue Longitude
Neighborhood Name,,,,,,,
Agincourt,5,5,5,5,5,5,5
"Alderwood, Long Branch",9,9,9,9,9,9,9
"Bathurst Manor, Wilson Heights, Downsview North",19,19,19,19,19,19,19
Bayview Village,2,2,2,2,2,2,2
"Bedford Park, Lawrence Manor East",20,20,20,20,20,20,20
...,...,...,...,...,...,...,...
"Willowdale, Willowdale West",6,6,6,6,6,6,6
Woburn,3,3,3,3,3,3,3
Woodbine Heights,16,16,16,16,16,16,16


How many categories of venues are there? 

In [28]:
len(Toronto_nei_venues['Venue Categorie'].unique())

265

In [48]:
len(Toronto_nei_venues['Borough Name'].unique())

10

# Analyzing Each Neighborhood

In [31]:
Toronto_venues_onehot = pd.get_dummies(Toronto_nei_venues[['Venue Categorie']] , prefix="", prefix_sep="")
Toronto_venues_onehot['Neighborhood Name'] = Toronto_nei_venues['Neighborhood Name']
new_col  = ['Neighborhood Name'] + list(Toronto_venues_onehot.columns[:-1])
Toronto_venues_onehot = Toronto_venues_onehot[new_col]
Toronto_venues_onehot.head()

,Neighborhood Name,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
Toronto_nei_venues.groupby(['Neighborhood Name','Borough Name']).count()

,,Neighborhood Latitude,Neighborhood Longitude,Venue Name,Venue Categorie,Venue Latitude,Venue Longitude
Neighborhood Name,Borough Name,,,,,,
Agincourt,Scarborough,5,5,5,5,5,5
"Alderwood, Long Branch",Etobicoke,9,9,9,9,9,9
"Bathurst Manor, Wilson Heights, Downsview North",North York,19,19,19,19,19,19
Bayview Village,North York,2,2,2,2,2,2
"Bedford Park, Lawrence Manor East",North York,20,20,20,20,20,20
...,...,...,...,...,...,...,...
"Willowdale, Willowdale West",North York,6,6,6,6,6,6
Woburn,Scarborough,3,3,3,3,3,3
Woodbine Heights,East York,16,16,16,16,16,16


### Let group venues by Neighborhoods

In [33]:
Toronto_venues_grouped = Toronto_venues_onehot.groupby('Neighborhood Name').mean().reset_index()
print("The shape is ",Toronto_venues_grouped.shape)
Toronto_venues_grouped.head()

The shape is  (97, 266)


,Neighborhood Name,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.052632,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


Let print each Neighborhood with its five most frequent categories

In [34]:
list(Toronto_venues_grouped.iloc[0,1: ].sort_values(ascending=False).reset_index().iloc[:5,0])

['Breakfast Spot',
 'Supermarket',
 'Badminton Court',
 'Skating Rink',
 'Sushi Restaurant']

In [35]:
for ind,nei in enumerate(Toronto_venues_grouped['Neighborhood Name']):
    print("_____"+nei+"_____")
    temp = Toronto_venues_grouped.iloc[ind,1:].sort_values(ascending= False).reset_index()[:5]
    temp.columns = ['venue','freq']
    print(temp)

_____Agincourt_____
              venue freq
0    Breakfast Spot  0.2
1       Supermarket  0.2
2   Badminton Court  0.2
3      Skating Rink  0.2
4  Sushi Restaurant  0.2
_____Alderwood, Long Branch_____
         venue      freq
0  Pizza Place  0.111111
1          Pub  0.111111
2  Coffee Shop  0.111111
3  Gas Station  0.111111
4          Gym  0.111111
_____Bathurst Manor, Wilson Heights, Downsview North_____
                       venue       freq
0                Coffee Shop   0.105263
1                       Bank   0.105263
2  Middle Eastern Restaurant  0.0526316
3           Sushi Restaurant  0.0526316
4                      Diner  0.0526316
_____Bayview Village_____
                        venue freq
0  Construction & Landscaping  0.5
1                       Trail  0.5
2                 Yoga Studio    0
3          Falafel Restaurant    0
4                  Donut Shop    0
_____Bedford Park, Lawrence Manor East_____
                     venue  freq
0       Italian Restaurant   0.1
1  

### Let consider the 10 most common venue for each neighborhood

We create a new dataframe for that

In [64]:
column = ['1st Most Common Venue','2nd Most Common Venue','3rd Most Common Venue','4th Most Common Venue',
         '5th Most Common Venue','6th Most Common Venue','7th Most Common Venue','8th Most Common Venue',
         '9th Most Common Venue','10th Most Common Venue']

Toronto_common_venues = pd.DataFrame([], columns=column)
Toronto_common_venues['Neighborhood Name'] = Toronto_venues_grouped['Neighborhood Name']
for ind,nei in enumerate(Toronto_venues_grouped['Neighborhood Name']) :
    
    val = list(Toronto_venues_grouped.iloc[ind,1:].sort_values(ascending = False).reset_index().iloc[0:10,0])
    Toronto_common_venues.loc[ind,:-1] = val
new_column = ['Neighborhood Name']+column
Toronto_common_venues = Toronto_common_venues[new_column] 
Toronto_common_venues.head()

,Neighborhood Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Supermarket,Badminton Court,Skating Rink,Sushi Restaurant,Field,Fast Food Restaurant,Fish & Chips Shop,Farmers Market,Donut Shop
1,"Alderwood, Long Branch",Pizza Place,Pub,Coffee Shop,Gas Station,Gym,Athletics & Sports,Convenience Store,Pharmacy,Sandwich Place,Farm
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Middle Eastern Restaurant,Sushi Restaurant,Diner,Gas Station,Supermarket,Sandwich Place,Deli / Bodega,Ice Cream Shop
3,Bayview Village,Construction & Landscaping,Trail,Yoga Studio,Falafel Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Farm
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Sandwich Place,Coffee Shop,Comfort Food Restaurant,Sports Club,Juice Bar,Restaurant,Thai Restaurant,Pub,Liquor Store


# Clustering : Segmentation of Neighborhoods in Toronto

K-Means segmentation algorithm

In [65]:
from sklearn.cluster import KMeans
# Drop the Neighborhood column since kmeans does not support string values
Toronto_venues_clustering = Toronto_venues_grouped.drop('Neighborhood Name', axis=1)

# kmeans computing with 5 clusters
kclusters = 5
Toronto_kmean = KMeans(n_clusters =kclusters).fit(Toronto_venues_clustering)

Toronto_kmean.labels_[:5]

array([2, 2, 2, 0, 2], dtype=int32)

Let reorganize the data with the labels

In [66]:
Toronto_common_venues.insert(0,'Cluster Labels',Toronto_kmean.labels_)
#Toronto_nei_venues.groupby(['Neighborhood Name','Borough Name','Neighborhood Latitude','Neighborhood Longitude']).count().reset_index()[['Neighborhood Name','Borough Name','Neighborhood Latitude','Neighborhood Longitude']]

Toronto_merged = Toronto_common_venues
col_list1 = ["Borough Name","Neighborhood Latitude",'Neighborhood Longitude']
Toronto_nei_venues_ord= Toronto_nei_venues.groupby(['Neighborhood Name']+col_list1).count().reset_index()
Toronto_merged[col_list1] = Toronto_nei_venues_ord[col_list1]

col_list2 = ["Borough Name","Neighborhood Name","Neighborhood Latitude",'Neighborhood Longitude']\
                +['Cluster Labels'] + list(Toronto_merged.columns[2:12])
Toronto_merged = Toronto_merged[col_list2]
Toronto_merged.head()

,Borough Name,Neighborhood Name,Neighborhood Latitude,Neighborhood Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,Agincourt,43.793930,-79.265694,2,Breakfast Spot,Supermarket,Badminton Court,Skating Rink,Sushi Restaurant,Field,Fast Food Restaurant,Fish & Chips Shop,Farmers Market,Donut Shop
1,Etobicoke,"Alderwood, Long Branch",43.600895,-79.540387,2,Pizza Place,Pub,Coffee Shop,Gas Station,Gym,Athletics & Sports,Convenience Store,Pharmacy,Sandwich Place,Farm
2,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.757394,-79.442394,2,Coffee Shop,Bank,Middle Eastern Restaurant,Sushi Restaurant,Diner,Gas Station,Supermarket,Sandwich Place,Deli / Bodega,Ice Cream Shop
3,North York,Bayview Village,43.780607,-79.376921,0,Construction & Landscaping,Trail,Yoga Studio,Falafel Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Farm
4,North York,"Bedford Park, Lawrence Manor East",43.735447,-79.417944,2,Italian Restaurant,Sandwich Place,Coffee Shop,Comfort Food Restaurant,Sports Club,Juice Bar,Restaurant,Thai Restaurant,Pub,Liquor Store


#### Let us see on the map the different clusters

In [70]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [103]:
Toronto_map_clusters = folium.Map(location=[latitude,longitude], zoom_start =10.3)

x = np.arange(kclusters)
ys = [i+x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

for bor,nei,lat,lng,clus in zip(Toronto_merged["Borough Name"], Toronto_merged["Neighborhood Name"], Toronto_merged["Neighborhood Latitude"],Toronto_merged["Neighborhood Longitude"],Toronto_merged["Cluster Labels"]):
    label = '{}, {}'.format(nei,bor)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat,lng],
                        radius = 5,
                        popup = label,
                        color = rainbow[clus],
                        fill = True,
                        fill_opacity = 0.7,
                        parse_html = False).add_to(Toronto_map_clusters)
Toronto_map_clusters